# Session 2: Monte Carlo Methods

## Colab Setup

In [ ]:
#uncomment only if you're running from google colab
# !git clone https://github.com/Datatouille/rl-workshop
# !mv rl-workshop/* .
# !ls

## Imports

In [1]:
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
#cross check with our solutions once you finish
from solutions.agents import MCAgent
from solutions.environments import Gridworld

## Fill in The Code

In [ ]:
# import numpy as np
# from collections import defaultdict
# import sys

# """
# Coding assignment order:
# 1. get_v
# 2. get_q
# 3. mc_control_q
# 4. mc_control_glie
# """

# class MCAgent:
#     def __init__(self, env, policy, gamma = 0.9, 
#                  start_epsilon = 0.9, end_epsilon = 0.1, epsilon_decay = 0.9):
#         self.env = env
#         self.n_action = len(self.env.action_space)
#         self.policy = policy
#         self.gamma = gamma
#         self.v = dict.fromkeys(self.env.state_space,0)
#         self.n_v = dict.fromkeys(self.env.state_space,0)
#         self.q = defaultdict(lambda: np.zeros(self.n_action))
#         self.n_q = defaultdict(lambda: np.zeros(self.n_action))
#         self.start_epsilon = start_epsilon
#         self.end_epsilon = end_epsilon
#         self.epsilon_decay = epsilon_decay
#     def get_epsilon(self,n_episode):
#         epsilon = max(self.start_epsilon * (self.epsilon_decay**n_episode),self.end_epsilon)
#         return(epsilon)
#     def get_v(self,start_state,epsilon = 0.):
#         episode = self.run_episode(start_state,epsilon)
#         """
#         Write the code to calculate the state value function of a state 
#         given a deterministic policy.
#         """
#         v=0
        
#         return(v)
#     def get_q(self, start_state, first_action, epsilon=0.):
#         episode = self.run_episode(start_state,epsilon,first_action)
#         """
#         Write the code to calculate the action function of a state 
#         given a deterministic policy.
#         """
#         q=0
#         return(q)
#     def select_action(self,state,epsilon):
#         probs = np.ones(self.n_action) * (epsilon / self.n_action)
#         best_action = self.policy[state]
#         probs[best_action] = 1 - epsilon + (epsilon / self.n_action)
#         action = np.random.choice(np.arange(self.n_action),p=probs)
#         return(action)
#     def print_policy(self):
#         for i in range(self.env.sz[0]):
#             print('\n----------')
#             for j in range(self.env.sz[1]):
#                 p=self.policy[(i,j)]
#                 out = self.env.action_text[p]
#                 print(f'{out} |',end='')
#     def print_v(self, decimal = 1):
#         for i in range(self.env.sz[0]):
#             print('\n---------------')
#             for j in range(self.env.sz[1]):
#                 out=np.round(self.v[(i,j)],decimal)
#                 print(f'{out} |',end='')
#     def run_episode(self, start, epsilon, first_action = None):
#         result = []
#         state = self.env.reset(start)
#         #dictate first action to iterate q
#         if first_action is not None:
#             action = first_action
#             next_state,reward,done = self.env.step(action)
#             result.append((state,action,reward,next_state,done))
#             state = next_state
#             if done: return(result)
#         while True:
#             action = self.select_action(state,epsilon)
#             next_state,reward,done = self.env.step(action)
#             result.append((state,action,reward,next_state,done))
#             state = next_state
#             if done: break
#         return(result)
#     def update_policy_q(self):
#         for state in self.env.state_space:
#             self.policy[state] = np.argmax(self.q[state])
#     def mc_predict_v(self,n_episode=10000,first_visit=True):
#         for t in range(n_episode):
#             traversed = []
#             e = self.get_epsilon(t)
#             transitions = self.run_episode(self.env.start,e)
#             states,actions,rewards,next_states,dones = zip(*transitions)
#             for i in range(len(transitions)):
#                 if first_visit and (states[i] not in traversed):
#                     traversed.append(states[i])
#                     self.n_v[states[i]]+=1
#                     discounts = np.array([self.gamma**j for j in range(len(transitions)+1)])
#                     self.v[states[i]]+= sum(rewards[i:]*discounts[:-(1+i)])
#         for state in self.env.state_space:
#             if state != self.env.goal:
#                 self.v[state] = self.v[state] / self.n_v[state]
#             else:
#                 self.v[state] = 0
    
#     def mc_predict_q(self,n_episode=10000,first_visit=True):
#         for t in range(n_episode):
#             traversed = []
#             e = self.get_epsilon(t)
#             transitions = self.run_episode(self.env.start,e)
#             states,actions,rewards,next_states,dones = zip(*transitions)
#             for i in range(len(transitions)):
#                 if first_visit and ((states[i],actions[i]) not in traversed):
#                     traversed.append((states[i],actions[i]))
#                     self.n_q[states[i]][actions[i]]+=1
#                     discounts = np.array([self.gamma**j for j in range(len(transitions)+1)])
#                     self.q[states[i]][actions[i]]+= sum(rewards[i:]*discounts[:-(1+i)])
#                 elif not first_visit:
#                     self.n_q[states[i]][actions[i]]+=1
#                     discounts = np.array([self.gamma**j for j in range(len(transitions)+1)])
#                     self.q[states[i]][actions[i]]+= sum(rewards[i:]*discounts[:-(1+i)])

#         #print(self.q,self.n_q)
#         for state in self.env.state_space:
#             for action in range(self.n_action):
#                 if state != self.env.goal:
#                     self.q[state][action] = self.q[state][action] / self.n_q[state][action]
#                 else:
#                     self.q[state][action] = 0
        
#     def mc_control_q(self,n_episode=10000,first_visit=True):
#         """
#         Write the code to perform Monte Carlo Control
#         Hint: You just need to do prediction then update the policy
#         """
#         pass
        
#     def mc_control_glie(self,n_episode=10000,first_visit=True,lr=0.):
#         """
#         Taking hints from the mc_predict_q and mc_control_q methods, write the code to
#         perform GLIE Monte Carlo control.
#         """
#         pass

![RL Framework](img/rl_framework.png)
Source: [Sutton and Barto](https://cdn.preterhuman.net/texts/science_and_technology/artificial_intelligence/Reinforcement%20Learning%20%20An%20Introduction%20-%20Richard%20S.%20Sutton%20,%20Andrew%20G.%20Barto.pdf)

## Solving Reinforcement Learning Problems - Monte Carlo Methods

There are two main approaches in solving reinforcement learning: **model-based** and **model-free** approaches. A model-based approach assumes that we have some or full knowledge of how our environment works whereas a model-free approach relies on our agent to explore the environment without any prior knowledge. 

In this workshop, we will focus on model-free approaches which usually involves two steps: evalulating the state or action value function based on the agent's interactions with the environment also known as **prediction problem** and changing the agent's policy to be closer to an optimal policy also known as **control problem**.

We start with the Monte Carlo Methods aka the trial-and-error-until-you-get-rich-or-broke methods.

![Monte Carlo](img/monte_carlo.jpg)

### Prediction Problem

In [2]:
#stochastic environment
env = Gridworld(wind_p=0.2)

#initial policy
policy = {(0, 0): 3,
          (0, 1): 3,
          (0, 2): 2,
          (1, 0): 3,
          (1, 1): 3,
          (1, 2): 0,
          (2, 0): 3,
          (2, 1): 0,
          (2, 2): 0}

#stochastic agent - epsilon greedy with decays
a = MCAgent(env, policy = policy, gamma = 0.9, 
            start_epsilon=0.9,end_epsilon=0.3,epsilon_decay=0.9)

print('Reward Grid')
env.print_reward()
print('\n')
print('Policy: Reach Goal ASAP')
a.print_policy()

Reward Grid

----------
0 |0 |0 |
----------
0 |-5 |5 |
----------
0 |0 |0 |

Policy: Reach Goal ASAP

----------
R |R |D |
----------
R |R |U |
----------
R |U |U |

#### Monte Carlo State Value Prediction

![Monte Carlo State Value Prediction](img/mc_predict_v.png)

In [3]:
a.mc_predict_v()
a.print_v()


---------------
-1.0 |0.8 |2.7 |
---------------
-3.6 |1.9 |0 |
---------------
-3.9 |-3.4 |2.5 |

#### Monte Carlo Action Value Prediction

![Monte Carlo Action Value Prediction](img/mc_predict_q.png)

In [4]:
a.mc_predict_q(first_visit=False)
print(f'\nActions: {env.action_text}')
for i in a.q: print(i,a.q[i])


Actions: ['U' 'L' 'D' 'R']
(2, 0) [-4.22439754 -4.71053496 -4.56914673 -3.83200812]
(0, 0) [-1.21149553 -1.55036833 -4.07535088 -0.43477837]
(0, 1) [-0.59506385 -1.97565183 -3.54684278  1.36483998]
(1, 0) [-1.80911499 -4.10394462 -4.51974484 -3.62873459]
(1, 1) [-0.83446812 -4.14362083 -3.8548538   3.30284643]
(2, 1) [-3.72587981 -4.22407659 -4.37450104  1.04162032]
(0, 2) [ 1.07652167 -0.62900575  3.20047428  1.34862222]
(2, 2) [ 3.31719125 -4.13110594  1.62764329  1.37645882]
(1, 2) [0. 0. 0. 0.]


### Control Problem

In [5]:
#stochastic environment
env = Gridworld(wind_p=0.2)

#initial policy
policy = {(0, 0): 3,
          (0, 1): 3,
          (0, 2): 2,
          (1, 0): 3,
          (1, 1): 3,
          (1, 2): 0,
          (2, 0): 3,
          (2, 1): 0,
          (2, 2): 0}

#stochastic agent - epsilon greedy with decays
a = MCAgent(env, policy = policy, gamma = 0.9, 
            start_epsilon=0.9,end_epsilon=0.3,epsilon_decay=0.9)

print('Reward Grid')
env.print_reward()
print('\n')
print('Policy: Reach Goal ASAP')
a.print_policy()

Reward Grid

----------
0 |0 |0 |
----------
0 |-5 |5 |
----------
0 |0 |0 |

Policy: Reach Goal ASAP

----------
R |R |D |
----------
R |R |U |
----------
R |U |U |

### All-visit Monte Carlo

**Coding Assignment** Implement `mc_control_q` function of `agent.py` using either all-visit or first-visit Monte Carlo.

In [7]:
#reset
a.policy = policy
a.q = defaultdict(lambda: np.zeros(a.n_action))
a.n_q = defaultdict(lambda: np.zeros(a.n_action))

a.mc_control_q(n_episode = 1000,first_visit=False)
a.print_policy()
print(f'\nActions: {env.action_text}')
for i in a.q: print(i,a.q[i])


----------
R |R |D |
----------
D |R |U |
----------
R |R |U |
Actions: ['U' 'L' 'D' 'R']
(2, 0) [-5.54833772 -1.85349972 -2.08578787 -0.73771261]
(1, 0) [-5.77592437 -6.59097517 -3.1292505  -5.7831827 ]
(0, 0) [-6.7395438  -6.42969785 -5.79815642 -1.18062795]
(0, 1) [-0.14403264 -7.28539313 -4.43318536  0.81095461]
(1, 1) [-3.07408888 -6.44085182 -1.10066989  3.3215208 ]
(0, 2) [ 2.285      -2.20036967  3.08081907  0.32828113]
(2, 1) [-4.0082097  -2.7446078  -0.81432703  1.42648518]
(2, 2) [ 3.29065298 -0.13140317  1.23347443  1.24559436]
(1, 2) [0. 0. 0. 0.]


#### First-visit Monte Carlo

In [9]:
#reset
a.policy = policy
a.q = defaultdict(lambda: np.zeros(a.n_action))
a.n_q = defaultdict(lambda: np.zeros(a.n_action))

a.mc_control_q(n_episode = 1000,first_visit=True)
a.print_policy()
print(f'\nActions: {env.action_text}')
for i in a.q: print(i,a.q[i])


----------
R |R |D |
----------
D |R |U |
----------
R |R |U |
Actions: ['U' 'L' 'D' 'R']
(2, 0) [-5.7812204  -2.2699994  -2.74398497 -0.85362256]
(2, 1) [-5.07340473 -2.43013695 -0.55785209  1.35426502]
(1, 1) [-5.27900541 -5.67040714  0.58638074  3.36698426]
(1, 0) [-6.30230831 -6.15461758 -2.17873572 -5.06438271]
(2, 2) [ 3.22276835 -0.79378102  0.89892963  1.54103934]
(0, 1) [-5.71977478 -5.94257766 -4.175052   -0.27523024]
(0, 2) [ 0.6560261  -5.24084074  3.21791465 -3.73845459]
(0, 0) [-5.9153525  -6.24478038 -6.15018493 -5.14217806]
(1, 2) [0. 0. 0. 0.]


#### Greedy within The Limit of Exploration 

![Greedy within The Limit of Exploration](img/mc_control_glie.png)

**Coding Assignment** Implement `mc_control_glie` function of `agent.py`

In [11]:
#reset
a.policy = policy
a.q = defaultdict(lambda: np.zeros(a.n_action))
a.n_q = defaultdict(lambda: np.zeros(a.n_action))

a.mc_control_glie(n_episode = 1000)
a.print_policy()
print(f'\nActions: {env.action_text}')
for i in a.q: print(i,a.q[i])


----------
R |R |D |
----------
D |R |U |
----------
R |R |U |
Actions: ['U' 'L' 'D' 'R']
(2, 0) [-2.95382854 -2.11975586 -2.02890926 -0.44634021]
(0, 0) [ -7.80549118  -9.01441985 -19.89607641  -0.28830266]
(0, 1) [-0.42184    -1.51044777 -6.307456    1.81574203]
(0, 2) [ 0.6072434  -0.06370055  3.82953243  0.75726958]
(1, 0) [-2.61118161 -2.22484694 -1.54050169 -3.65241881]
(1, 1) [-1.7642455  -3.56930715 -0.6064638   3.3816412 ]
(1, 2) [0. 0. 0. 0.]
(2, 1) [-2.88671386 -1.69264483 -0.60669012  1.553508  ]
(2, 2) [ 3.40147724 -0.26747147  1.68730766  1.50660898]


#### GLIE with Constant Learning Rate

![GLIE with constant learning rate](img/mc_control_glie_constant.png)

In [12]:
#reset
a.policy = policy
a.q = defaultdict(lambda: np.zeros(a.n_action))
a.n_q = defaultdict(lambda: np.zeros(a.n_action))

a.mc_control_glie(n_episode = 1000, lr=0.1)
a.print_policy()
print(f'\nActions: {env.action_text}')
for i in a.q: print(i,a.q[i])


----------
R |R |D |
----------
D |R |U |
----------
R |R |U |
Actions: ['U' 'L' 'D' 'R']
(2, 0) [-2.87203657 -2.58126939 -2.31964153 -1.80949397]
(0, 0) [-2.11089394 -2.31134566 -3.19471733 -1.08518698]
(0, 1) [-0.24222561 -2.27679431 -2.94693795  1.23517985]
(0, 2) [ 0.23108981 -0.99888656  3.47545143 -0.71247279]
(1, 0) [-3.14516001 -3.03619437 -2.16901959 -4.23461649]
(1, 1) [-1.30635314 -3.58962232  0.11025162  2.68320716]
(1, 2) [0. 0. 0. 0.]
(2, 1) [-4.21192083 -2.66302766 -0.6495667   0.14668219]
(2, 2) [ 2.6184096  -1.07123348  2.21696245  1.45197172]


## Challenges

* What are some other ways of solving reinforcement learning problems? How are they better or worse than Monte Carlo methods e.g. performance, data requirements, etc.?
* Solve at least one of the following OpenAI gym environments with discrete states and actions:
    * FrozenLake-v0
    * Taxi-v2
    * Blackjack-v0